## Install Dependency

In [1]:
%pip install sentence-transformers pandas faiss-cpu openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\SaeedM\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Configuration Setup (JSON)

In [2]:
import json

with open('config.json', 'r', encoding='utf-8') as f:
    config = json.load(f)

print("Loaded configuration:")
print(json.dumps(config, indent=2, ensure_ascii=False))

Loaded configuration:
{
  "embedding_model": "paraphrase-multilingual-MiniLM-L12-v2",
  "lm_studio": {
    "base_url": "http://192.168.220.1:3000/v1/",
    "api_key": "lm-studio"
  },
  "model_identifier": "dorna2-llama3.1-8b-instruct",
  "top_k": 5,
  "system_template": "شما یک دستیار هوشمند فارسی هستید. با استفاده از اطلاعات داده شده به سوالات کاربر پاسخ دقیق و مفصل بدهید.",
  "retrieval_method": ""
}


## Embedding & Indexing

In [3]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import pickle

df = pd.read_csv("university_info.csv")
df.fillna("", inplace=True)

def create_document(row):
    return (
        f"نام و نام خانوادگی: {row['نام و نام خانوادگی']}. "
        f"عنوان سازمانی: {row['عنوان سازمانی']}. "
        f"واحد سازمانی: {row['واحد سازمانی']}. "
        f"شماره داخلی: {row['شماره داخلی']}. "
        f"شماره مستقیم: {row['شماره مستقیم']}. "
        f"ایمیل: {row['ایمیل']}."
    )

csv_documents = df.apply(create_document, axis=1).tolist()

with open("info.txt", "r", encoding="utf-8") as f:
    text_content = f.read()
txt_paragraphs = [p.strip() for p in text_content.split("\n\n") if p.strip() != ""]

all_documents = csv_documents + txt_paragraphs

#  Embedding Generation
embed_model = SentenceTransformer(config['embedding_model'])
embeddings = embed_model.encode(all_documents, show_progress_bar=True)

#  Indexing
embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings, dtype=np.float32))

# Save index and documents
faiss.write_index(index, "vector_index.faiss")
with open("documents.pkl", "wb") as f:
    pickle.dump(all_documents, f)

print(f"Index created with {len(all_documents)} documents")

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

Index created with 1085 documents


## Retriever Implementation

In [4]:
class PersianRetriever:
    def __init__(self, index_path, documents_path):
        self.index = faiss.read_index(index_path)
        with open(documents_path, "rb") as f:
            self.documents = pickle.load(f)
        self.embed_model = SentenceTransformer(config['embedding_model'])
    
    def get_relevant_documents(self, query, top_k=config["top_k"]):
        query_embed = self.embed_model.encode(query)
        distances, indices = self.index.search(np.array([query_embed], dtype=np.float32), top_k)
        print("distances from query :", distances)
        return [self.documents[i] for i in indices[0] if i < len(self.documents)]

retriever = PersianRetriever(
    index_path="vector_index.faiss",
    documents_path="documents.pkl"
)

## LM Studio Connection

In [5]:
from openai import OpenAI

client = OpenAI(
    base_url=config['lm_studio']['base_url'],
    api_key=config['lm_studio']['api_key']
)

class PersianLLM:
    def __init__(self, model_identifier, system_template=config['system_template']):
        self.model = model_identifier
        self.system_template = system_template
    
    def generate(self, context, user_query, temperature=0.7):
        try:
            completion = client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": self.system_template},
                    {"role": "user", "content": f"متن زمینه:\n{context}\n\nسوال: {user_query}"}
                ],
                temperature=temperature,
            )
            return completion.choices[0].message.content
        except Exception as e:
            print(e)
            return "Error in generate function!"


## Query Transformation Methods

In [6]:
def multi_query_generation(user_query, llm=None):
    variations = [
        f"{user_query}",
        f"لطفاً توضیح دهید: {user_query}",
        f"مفهوم اصلی {user_query} چیست؟",
        f"ارائه مثال برای {user_query}",
        f"خلاصه‌ای درباره {user_query}"
    ]
    
    if llm:
        prompt = f"سه روش مختلف برای پرسش درباره این موضوع ایجاد کن: {user_query}"
        llm_variations = llm.invoke(prompt).split('\n')[:3]
        variations.extend(llm_variations)
    
    return variations

def multi_query_retrieval(retriever, user_query, llm=None, top_k=5):
    queries = multi_query_generation(user_query, llm)
    results = []
    for query in queries:
        results.extend(retriever.get_relevant_documents(query, top_k=top_k))
    return list(set(results))[:top_k]

def rag_fusion(retriever, queries, top_k=5):
    from collections import defaultdict
    
    doc_scores = defaultdict(float)
    for query in queries:
        docs = retriever.get_relevant_documents(query, top_k=top_k)
        for i, doc in enumerate(docs):
            # Use reciprocal rank fusion
            doc_scores[doc] += 1/(i + 1)
    
    sorted_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
    return [doc for doc, score in sorted_docs[:top_k]]

def step_back_retrieval(retriever, user_query, top_k=5):
    initial_results = retriever.get_relevant_documents(user_query, top_k=top_k)
    if not initial_results:
        return retriever.get_relevant_documents(f"موضوع کلی: {user_query}", top_k=top_k)
    return initial_results

def hyde_retrieval(retriever, user_query, llm, top_k=5):
    synthetic_answer = llm.invoke(f"فرض کنید متخصص هستید، خلاصه‌ای درباره {user_query} ارائه دهید.")
    return retriever.get_relevant_documents(synthetic_answer, top_k=top_k)

## Functions

In [7]:
def retrieve_answer(user_query, retriever, llm, method="default", top_k=config["top_k"]):
    method = config['retrieval_method']
    if method == "multi_query":
        return multi_query_retrieval(retriever, user_query, llm, top_k)
    elif method == "rag_fusion":
        queries = multi_query_generation(user_query, llm, top_k)
        return rag_fusion(retriever, queries)
    elif method == "step_back":
        return step_back_retrieval(retriever, user_query, top_k)
    elif method == "hyde":
        return hyde_retrieval(retriever, user_query, llm, top_k)
    else:
        return retriever.get_relevant_documents(user_query, top_k)

def format_context(documents, max_length=4000):
    context = []
    total_len = 0
    for doc in documents:
        doc_len = len(doc.split())
        if total_len + doc_len > max_length:
            break
        context.append(doc)
        total_len += doc_len
    return "\n\n".join(context)

def safe_serialize(obj):
    if isinstance(obj, (str, int, float, bool)):
        return obj
    elif isinstance(obj, (list, tuple)):
        return [safe_serialize(item) for item in obj]
    elif isinstance(obj, dict):
        return {k: safe_serialize(v) for k, v in obj.items()}
    else:
        return str(obj)

## Main Execution Flow

In [8]:

llm = PersianLLM(
    model_identifier=config['model_identifier'],
    system_template=config['system_template']
)

while True:
    try:
        user_query = input("\nسوال خود را وارد کنید (یا 'خروج' برای پایان): ").strip()
        if user_query.lower() in ['exit', 'خروج']:
            break
    
        docs = retrieve_answer(user_query, retriever, llm, method=config["retrieval_method"])
        context = format_context(docs)
        
        if not context:
            print("No near chunk found.")
            continue
        
        answer = llm.generate(context, user_query)
        print(answer)
        
        # Save results
        output = {
            "query": user_query,
            "retrieved_documents": safe_serialize(docs),
            "generated_answer": safe_serialize(answer)
        }
        
        with open('output.json', 'w', encoding='utf-8') as f:
            json.dump(output, f, ensure_ascii=False, indent=2)
        
    except KeyboardInterrupt:
        break
    except Exception as e:
        print(e)


distances from query : [[10.853364  11.7009945 12.542284  12.542995  12.601054 ]]
شما به دنبال شماره داخلی دکتر کریمی هستید، که در متن زمینه ذکر نشده است. با این حال، می‌توانم اطلاعات تماس سایر اعضای هیأت علمی دانشکده دامپزشکی را برای شما فراهم کنم.
آیا می‌خواهید شماره تلفن یا ایمیل سایر اعضای هیأت علمی را نیز داشته باشید؟
distances from query : [[10.601017 10.870323 10.934126 10.952851 11.091666]]
محمد رضائی، عضو هیأت علمی گروه الهیات و دانشکده علوم انسانی، شماره داخلی 6147 را دارد.
distances from query : [[8.165365 8.383644 8.399237 8.581251 8.697864]]
متاسفانه با اطلاعات داده شده نمی‌توانم به سوال شما پاسخ دهم. لطفا جزئیات بیشتری در مورد "دکتر رضا محمدی" ارائه دهید تا بتوانم به سوال شما پاسخ دهم. ممکن است این فرد در یکی از واحدهای سازمانی ذکر شده کار کند و شماره داخلی او را بتوان پیدا کرد.
distances from query : [[ 9.192681  11.2084875 11.387823  11.559641  11.630263 ]]
برای تماس با این افراد، می‌توانید از روش‌های زیر استفاده کنید:
- برای تماس از داخل دانشگاه:
- با ۹ در کنار شماره د